In [ ]:
!pip3 install transformers

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler
import numpy as np
import os
import sys
import json
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

# 1. load the list of ingredients and appropriate class labels associated
# 2. train pre-trained BERT classifier
# 3. curate class labels for existing pairs, and save as file

class RecipeIngredientsDataset(Dataset):

    def __init__(self, filename, cuisine_to_idx):
        with open(filename, 'r') as f:
            data = json.load(f)
            self.data = [{'recipe_id': k, 'ingredients': v} for k, v in data.items()]
            self.tokenizer = AutoTokenizer.from_pretrained(bert_model)
            
            self.cuisine_to_idx = cuisine_to_idx
            self.idx_to_cuisine = {v:k for k,v in self.cuisine_to_idx.items()}

    def __getitem__(self, index):
        ingredients = self.data[index]['ingredients']
        recipe_id = self.data[index]['recipe_id']
        tokenized = self.tokenizer(' '.join(ingredients), padding='max_length', truncation=True)
        tokenized = {key: torch.tensor(val) for key, val in tokenized.items()}

        return tokenized, recipe_id

    def __len__(self):
        return len(self.data)


bert_model = 'distilbert-base-uncased'

class WeakLabelDataset(Dataset):
    
    def __init__(self, filename, partition='train'):
        with open(filename, 'r') as f:
            self.data = json.load(f)
            self.partition = partition
            self.tokenizer = AutoTokenizer.from_pretrained(bert_model)
            self.cuisine_to_idx = {}

            if self.partition == 'train':
                self.get_class_mapping()

    def get_class_mapping(self):
        cuisines = set()
        for i, item in enumerate(self.data):
            cuisines.add(item['cuisine'])
        cuisines = list(cuisines)
        cuisines.sort()
        print(f"CUISINES: {cuisines}")

        self.cuisine_to_idx = {}
        for i, cuisine in enumerate(cuisines):
            self.cuisine_to_idx[cuisine] = i

    def __len__(self):
        return len(self.data)

    def prepare_data(self, list_of_ingredients):
        return self.tokenizer(' '.join(list_of_ingredients), padding='max_length', truncation=True)

    def __getitem__(self, index):
        
        if self.partition == 'train':
            item = {key: torch.tensor(val) for key, val in self.prepare_data(self.data[index]['ingredients']).items()}
            item['labels'] = self.cuisine_to_idx[self.data[index]['cuisine']]
            
            return item

        else:
            item = {key: torch.tensor(val) for key, val in self.prepare_data(self.data[index]['ingredients']).items()}

            return item

class WeakLabels():

    def __init__(self, train_file, test_file):
        self.train_dataset = WeakLabelDataset(train_file)
    
    def train(self):
        self.train_dataloader = DataLoader(self.train_dataset, shuffle=True, batch_size=50, num_workers=4, drop_last=True)
        
        self.device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
        self.model = AutoModelForSequenceClassification.from_pretrained(bert_model, num_labels=len(self.train_dataset.cuisine_to_idx))

        optimizer = torch.optim.Adam(self.model.parameters(), lr=5e-5)
        num_epochs = 2
        num_training_steps = num_epochs * len(self.train_dataloader)
        lr_scheduler = get_scheduler(
            name='linear',
            optimizer=optimizer,
            num_warmup_steps=0,
            num_training_steps=num_training_steps
        )
        self.model = self.model.to(self.device)

        progress_bar = tqdm(range(num_training_steps))

        for epoch in tqdm(range(num_epochs)):
            for batch in self.train_dataloader:
                batch = {k: v.to(self.device) for k, v in batch.items()}
                outputs = self.model(**batch)
                loss = outputs.loss
                loss.backward()

                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()

                progress_bar.update(1)
        
        torch.save(self.model.state_dict(), 'latest_model_dict.pt')


    def curate_labels(self):
        
        self.model.eval()

        # create a smaller dataset class reading from the cleaned_ingredients JSON file
        # pass it through the same pipeline as the current data loader
        # obtain the outputs for each recipe and map it with the recipe ID
        # save it into a file with the recipe id with label

        target_dataset = RecipeIngredientsDataset('../../recipe1M_layers/cleaned_ingredients.json', self.train_dataset.cuisine_to_idx)
        target_dataloader = DataLoader(target_dataset, batch_size=1, shuffle=False)
        
        final_output = {}

        for (item, recipe_id) in target_dataloader:
            item = {k: v.to(self.device) for k, v in item.items()}
            with torch.no_grad():
                outputs = self.model(**item)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)

            print(f"PREDICTIONS: {predictions}, LEN: {predictions.shape}")

            final_output[recipe_id] = target_dataset.idx_to_cuisine[int(predictions[0].detach().cpu())]

        json.dump(final_output, open('recipe-cuisine.json', 'w'))



weakLabels = WeakLabels('train.json', '')
weakLabels.train()
# weakLabels.curate_labels()

In [ ]:
!cp /content/model_dict.pt '/content/drive/MyDrive/Multimodal Project'

In [10]:
import torch.utils.data as data
from PIL import Image
import os
import pickle
import numpy as np
import lmdb
import torch
import sys
import json

from torchvision import transforms


def default_loader(path):
    try:
        im = Image.open(path).convert('RGB')
        return im
    except:
        print('This is stupid, image not opening')
        return Image.new('RGB', (224, 224), 'white')
       
class ImageLoader(data.Dataset):
    def __init__(self, img_path, transform=None, target_transform=None,
                 loader=default_loader, square=False, data_path=None, partition=None, sem_reg=None, clean_ingredients='', clean_layers='', image_only=False, title_only=False, cuisine_classes=''):

        if data_path == None:
            raise Exception('No data path specified.')

        if partition is None:
            raise Exception('Unknown partition type %s.' % partition)
        else:
            self.partition = partition

        self.env = lmdb.open(os.path.join(data_path, partition + '_lmdb'), max_readers=1, readonly=True, lock=False,
                             readahead=False, meminit=False)

        with open(os.path.join(data_path, partition + '_keys.pkl'), 'rb') as f:
            self.ids = pickle.load(f)

        self.square = square
        self.imgPath = img_path
        self.mismtch = 0.8
        self.maxInst = 20

        if sem_reg is not None:
            self.semantic_reg = sem_reg
        else:
            self.semantic_reg = False

        self.transform = transform
        self.target_transform = target_transform
        self.loader = loader
        
        with open(clean_ingredients, 'r') as f:
            self.ingredients = json.load(f)
        
        with open(clean_layers, 'r') as f:
            self.recipes = json.load(f)

        self.image_only = image_only
        self.title_only = title_only
        
        if cuisine_classes:
            self.cuisine_classes = json.load(open('recipe-cuisine-test.json', 'r'))

    def __getitem__(self, index):
        recipId = self.ids[index]
        # we force 80 percent of them to be a mismatch
        if self.partition == 'train':
            match = np.random.uniform() > self.mismtch
        elif self.partition == 'val' or self.partition == 'test':
            match = True
        else:
            raise 'Partition name not well defined'

        if self.title_only == True:
            title = self.recipes[recipId]['title']
            if self.cuisine_classes:
                return title, self.cuisine_classes.get(recipId, '')
            return title

        target = match and 1 or -1

        with self.env.begin(write=False) as txn:
            serialized_sample = txn.get(self.ids[index].encode('latin1'))
        sample = pickle.loads(serialized_sample,encoding='latin1')
        imgs = sample['imgs']

        # image
        if target == 1:
            if self.partition == 'train':
                # We do only use the first five images per recipe during training
                imgIdx = np.random.choice(range(min(5, len(imgs))))
            else:
                imgIdx = 0

            loader_path = [imgs[imgIdx]['id'][i] for i in range(4)]
            loader_path = os.path.join(*loader_path)
            # path = os.path.join(self.imgPath, self.partition, loader_path, imgs[imgIdx]['id'])
            path = os.path.join(self.imgPath, loader_path, imgs[imgIdx]['id'])
        else:
            # we randomly pick one non-matching image
            all_idx = range(len(self.ids))
            rndindex = np.random.choice(all_idx)
            while rndindex == index:
                rndindex = np.random.choice(all_idx)  # pick a random index

            with self.env.begin(write=False) as txn:
                serialized_sample = txn.get(self.ids[rndindex].encode('latin1'))

            rndsample = pickle.loads(serialized_sample,encoding='latin1')
            rndimgs = rndsample['imgs']

            if self.partition == 'train':  # if training we pick a random image
                # We do only use the first five images per recipe during training
                imgIdx = np.random.choice(range(min(5, len(rndimgs))))
            else:
                imgIdx = 0

            loader_path = [rndimgs[imgIdx]['id'][i] for i in range(4)]
            loader_path = os.path.join(*loader_path)
            path = os.path.join(self.imgPath, loader_path, rndimgs[imgIdx]['id'])
            # path = self.imgPath + rndimgs[imgIdx]['id']

        if self.image_only:
            img = self.loader(path)
            if self.square:
                img = img.resize(self.square)
            if self.transform is not None:
                img = self.transform(img)
            
            return img

        # instructions
        instrs = sample['intrs']
        itr_ln = len(instrs)
        t_inst = np.zeros((self.maxInst, np.shape(instrs)[1]), dtype=np.float32)
        t_inst[:itr_ln][:] = instrs
        instrs = torch.FloatTensor(t_inst)

        # ingredients
        ingrs = sample['ingrs'].astype(int)
        ingrs = torch.LongTensor(ingrs)
        igr_ln = max(np.nonzero(sample['ingrs'])[0]) + 1

        # load image
        print(f"IMAGE PATH: {path}")
        img = self.loader(path)

        if self.square:
            img = img.resize(self.square)
        if self.transform is not None:
            img = self.transform(img)
        if self.target_transform is not None:
            target = self.target_transform(target)

        rec_class = sample['classes'] - 1
        rec_id = self.ids[index]

        if target == -1:
            img_class = rndsample['classes'] - 1
            img_id = self.ids[rndindex]
        else:
            img_class = sample['classes'] - 1
            img_id = self.ids[index]

        # read from JSON files and load text ingredients + instructions
        ingredients = self.ingredients[recipId]
        title = self.recipes[recipId]['title']
        instructions = self.recipes[recipId]['instructions']

        # output
        if self.partition == 'train':
            if self.semantic_reg:
                return [img, instrs, itr_ln, ingrs, igr_ln, title, ingredients, instructions], [target, img_class, rec_class]
            else:
                return [img, instrs, itr_ln, ingrs, igr_ln, title, ingredients, instructions], [target]
        # Adding the cuisine information only here, since works only for test as of now... Will update as I run it on the entire dataset
        else:
            if self.semantic_reg:
                return [img, instrs, itr_ln, ingrs, igr_ln, title, ingredients, instructions], [target, img_class, rec_class, img_id, rec_id]
            elif len(self.cuisine_classes):
                return [img, instrs, itr_ln, ingrs, igr_ln, title, ingredients, instructions, self.cuisine_classes[recipId]], [target, img_id, rec_id]
            else:
                return [img, instrs, itr_ln, ingrs, igr_ln, title, ingredients, instructions], [target, img_id, rec_id]

    def __len__(self):
        return len(self.ids)

In [ ]:
import numpy as np
from PIL import Image
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torch.utils.data as data
import sys
import matplotlib.pyplot as plt
plt.ion()


# HELPER FUNCTIONS ------------------------------------------------------------------------------------------------------------------------------------------

def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.detach().cpu().numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}



# MAIN LOGIC ------------------------------------------------------------------------------------------------------------------------------------------
""""""
data_loader = data.DataLoader(
    ImageLoader(
        'test/',
        transform=transforms.Compose([
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]),
        data_path='test',
        partition='test',
        clean_ingredients='/content/drive/MyDrive/Multimodal Project/cleaned_json/cleaned_ingredients.json',
        clean_layers='/content/drive/MyDrive/Multimodal Project/cleaned_json/cleaned_layers.json',
        image_only=False,
        title_only=True),
    batch_size=128,
    shuffle=False
)

In [ ]:
model_conv = models.resnet50(pretrained=True)
model_conv = torch.nn.Sequential(*list(model_conv.children())[:-1])
model_conv = model_conv.to(device)
for params in model_conv.parameters():
    params.requires_grad = False

from tqdm import tqdm
print(len(data_loader))
outputs = []
for i, images in tqdm(enumerate(data_loader)):
    images = torch.tensor(images)
    images = images.to(device)

    output = torch.squeeze(model_conv(images))
    outputs.append(output)


In [ ]:
outputs = outputs[:-1]
outputs = torch.stack(outputs, dim=0)
outputs = outputs.detach().cpu().numpy()

print(outputs.shape)

In [ ]:
outputs = np.vstack(outputs)
print(outputs.shape)

(51328, 2048)


In [ ]:
np.save('/content/drive/MyDrive/Multimodal Project/image_encodings.npy', outputs)

In [2]:
!mkdir test && cd test/
!wget http://data.csail.mit.edu/im2recipe/test.tar
!tar -xvf test.tar
!cd ../

# !wget http://data.csail.mit.edu/im2recipe/recipe1M_images_test.tar
# !tar -xvf recipe1M_images_test.tar

--2022-10-11 16:56:13--  http://data.csail.mit.edu/im2recipe/test.tar
Resolving data.csail.mit.edu (data.csail.mit.edu)... 128.52.129.40
Connecting to data.csail.mit.edu (data.csail.mit.edu)|128.52.129.40|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5455892480 (5.1G) [application/octet-stream]
Saving to: ‘test.tar’

test.tar            100%[===================>]   5.08G   119MB/s    in 43s     

2022-10-11 16:56:56 (120 MB/s) - ‘test.tar’ saved [5455892480/5455892480]

test_lmdb/
test_lmdb/data.mdb
test_lmdb/lock.mdb
test_keys.pkl


In [3]:
import numpy as np
from PIL import Image
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torch.utils.data as data
import sys
import matplotlib.pyplot as plt
plt.ion()

In [5]:
outputs = np.load('/content/drive/MyDrive/Multimodal Project/image_encodings.npy')[:10000]

In [6]:
from sklearn.manifold import TSNE
import pandas as pd
import seaborn as sns

num_components = 2
tsne = TSNE(num_components)

In [7]:
tsne_result = tsne.fit_transform(outputs)
tsne_result.shape

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


(10000, 2)

In [11]:
title_data_loader = data.DataLoader(
    ImageLoader(
        'test/',
        transform=transforms.Compose([
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]),
        data_path='test',
        partition='test',
        clean_ingredients='/content/drive/MyDrive/Multimodal Project/cleaned_json/cleaned_ingredients.json',
        clean_layers='/content/drive/MyDrive/Multimodal Project/cleaned_json/cleaned_layers.json',
        image_only=False,
        title_only=True,
        cuisine_classes='recipe-cuisine-test.json'),
    batch_size=128,
    shuffle=False,
    drop_last=True
)

In [12]:
from tqdm import tqdm

titles = []
cuisines = []
for i, (title, cuisine) in tqdm(enumerate(title_data_loader)):
    cuisines.extend(list(cuisine))
    titles.extend(list(title))

401it [00:00, 2690.61it/s]


In [21]:
titles = titles[:10000]
cuisines = cuisines[:10000]

tsne_result_df = pd.DataFrame({'tsne_1': tsne_result[:,0], 'tsne_2': tsne_result[:,1], 'label': cuisines, 'food_title': titles})
import plotly.express as px

fig = px.scatter(tsne_result_df, x='tsne_1', y='tsne_2', color='label', hover_data=['label', 'food_title'], width=800, height=600)
fig.show()

In [19]:
for recipe, cuisine in zip(titles[10:25], cuisines[10:25]):
    print(f"{recipe} & {cuisine} \\\\")

Israeli Couscous Risotto with Shiitakes & moroccan \\
Gluten-free Vanilla Bean Cupcakes & southern_us \\
Quick and easy tuna casserole & italian \\
Fast Hammy Grits Recipe & southern_us \\
Bachelor Button Cookies & british \\
Italian Vegetarian Patties & italian \\
Tuna Sandwich & thai \\
Super easy fail proof budget cake & mexican \\
White Beans With Lemon, Garlic and Rosemary & italian \\
Mini Taco Salad Appetizers & mexican \\
Honeydew and Lime Soup & mexican \\
Hayashi Rice & spanish \\
red ground beef and rice & southern_us \\
Ants Climbing a Tree & chinese \\
Broccoli Puff & italian \\
